In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import re

In [2]:
os.chdir('../')
las_vegas = pd.read_csv('data/Las_Vegas_user_groupby_review.csv')
las_vegas_all_reviews = pd.read_csv('data/Las_Vegas_review.csv')
#reviews = pd.read_csv('data/reviews.csv')
business = pd.read_csv('data/business.csv')

In [15]:
business = pd.read_csv('data/business.csv')
business.head()

,Unnamed: 0,name,business_id,city,categories
0,0,The Range At Lake Norman,f9NumwFMBDn751xgFiRbNA,Cornelius,"Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh..."
1,1,"Carlos Santo, NMD",Yzvjg0SayhoZgCljUJRF9Q,Scottsdale,"Health & Medical, Fitness & Instruction, Yoga,..."
2,2,Felinus,XNoUzKckATkOD1hP6vghZg,Montreal,"Pets, Pet Services, Pet Groomers"
3,3,Nevada House of Hose,6OAZjbxqM5ol29BuHsil3w,North Las Vegas,"Hardware Stores, Home Services, Building Suppl..."
4,4,USE MY GUY SERVICES LLC,51M2Kk903DFYI6gnB5I6SQ,Mesa,"Home Services, Plumbing, Electricians, Handyma..."


In [ ]:
cities = ['Las Vegas','Phoenix','Toronto']
business.dropna(inplace = True)
restaurants_business = business[business.categories.str.contains("Restaurants")]

In [ ]:
business_city = restaurants_business[restaurants_business.city == 'Las Vegas']
bus_rev_city = business_city.merge(reviews, on = 'business_id')

In [ ]:
bus_rev_city = bus_rev_city.drop(columns = ['Unnamed: 0'])

### Preprocess for TF-IDF

In [6]:
las_vegas_all_reviews

,name,business_id,city,categories,review_id,text,stars,user_id
0,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",WUAt5-krh075Ie44np8Hew,So why would I be giving a Fast Food chain loc...,5.0,YDkC5VVT8s9NlIZtAr8NUA
1,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",S9vIs-je49jqbhpWSQ8yHg,I come here bout 3x's a mo. & I just can't do ...,2.0,mxtalrN7VHVQwGjv8ln5yg
2,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",i2Soid_X8XsSTyIrPOst6g,WORST experience EVER!!!!! never have i ate an...,1.0,xammeA3ftpFoAN13v5Blyg
3,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",uDYGaAOUgo7SLuH5nUZs3w,Hot fresh food usually. Staff seems to turn o...,4.0,gQhFacMGI41mfVXqw1E7bQ
4,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",Xg7HuBvuZHTBog1G_2Wk0w,Beer battered cod fish seems really rubbery. I...,3.0,0PXhJMztE3ijzQPjiwPERg
...,...,...,...,...,...,...,...,...
1503839,Island Buffet,wjZ12shpUWAy4Boy0G5z1g,Las Vegas,"Restaurants, Buffets",3TD_bRi98t5UDLuesY66Hw,Came to Vegas with the fam and some other fami...,1.0,UjBiVw0WFmEbgK6qgap3QA
1503840,Island Buffet,wjZ12shpUWAy4Boy0G5z1g,Las Vegas,"Restaurants, Buffets",IeowybrKAmmf9njf7pa1rA,So there are alot of pretty bad reviews but I ...,3.0,F5YcdVMqhDAfhGZsMCXDTA
1503841,Island Buffet,wjZ12shpUWAy4Boy0G5z1g,Las Vegas,"Restaurants, Buffets",r7maiBTmlfFRcaLAmwY0Aw,All the buffets are pretty much the same. Ove...,2.0,CPLndzPfGfJzjqjzXCr9_g
1503842,Island Buffet,wjZ12shpUWAy4Boy0G5z1g,Las Vegas,"Restaurants, Buffets",UPn6fAXxo7SEaQL4xkUgFA,Don't come here for dinner. You can see the b...,1.0,pSoe4XBftnJiTgG1fPw52g


In [6]:
def preprocess_group_user_review(review_df):
    user_df = review_df[['text','categories','user_id']].groupby('user_id').agg({'text':list,
                                                                            'categories':lambda column: ' '.join(column)}
                                                                          ).reset_index()
    user_df.text = user_df.text.apply(lambda t: "".join(re.sub(r'[^\w\s]',' ',str(t))).replace("\n"," "))
    return user_df

In [7]:
def preprocess_group_restaurant_review(review_df, business_df):
    restaurant_df = review_df[['text','stars','business_id']].groupby('business_id').agg({'text':list,
                                                                                    'stars':np.mean}
                                                                                   ).reset_index()
    restaurant_df.text = restaurant_df.text.apply(lambda t: "".join(re.sub(r'[^\w\s]',' ',str(t))).replace("\n"," "))
    return restaurant_df.merge(business_df[['name', 'business_id']], on = 'business_id')

In [9]:
test_all_reviews = las_vegas_all_reviews.head(1000000)

In [10]:
joined_user = preprocess_group_user_review(test_all_reviews)
joined_user

,user_id,text,categories
0,---1lKK3aKOuomHnwAkAow,I like the Lamb Tandoori dish they do a lot ...,"Pakistani, Restaurants, Indian Local Flavor, R..."
1,---udAKDsn0yQXmzbWQNSw,We were visiting from SoCal and was in the m...,"Restaurants, Filipino Food, Restaurants, Thai"
2,--0OjK-YhWtnTo4zO9llqw,The hotel is old and not well maintained W...,"Beauty & Spas, Hotels, Restaurants, Lounges, R..."
3,--0sXNBv6IizZXuV-nl0Aw,This place is awesome The pasta is homemade...,"Noodles, Restaurants, Italian"
4,--0zxhZTSLZ7w1hUD2bEwA,On our way to Utah we found this place on ye...,"Steakhouses, Food, Grocery, American (Traditio..."
...,...,...,...
417184,zzvqVZTYs5VKxPc-IkRQ4A,Went here for lunch cause I happened to be o...,"Restaurants, Mexican"
417185,zzxZoMmjbUjXcWZzrE3PIw,Thought we d try this place after passing i...,"Caterers, Italian, Event Planning & Services, ..."
417186,zzxbZgRaVD66oLRiF5LCxA,Staff was friendly but the room they put us ...,"Nightlife, Hotels & Travel, Arts & Entertainme..."
417187,zzyzP--p98zZtx359NvgGA,I love middle eastern food Love their chick...,"Mediterranean, Restaurants, Middle Eastern"


In [8]:
test_joined_restaurant = preprocess_group_restaurant_review(las_vegas_all_reviews, business)
test_joined_restaurant

,business_id,text,stars,name
0,--9e1ONYQuAa-CB_Rrw7Tw,On yelp 5 stars Woohoo as good as it gets...,4.117841,Delmonico Steakhouse
1,--q7kSBRb0vWC8lSkXFByA,This is a fun spot to watch games and eat go...,4.000000,Double Play Sports Bar
2,-0RkJ_uIduNLWQrphbADRw,Amazing food and service So grateful for th...,4.132653,Rooster Boy Cafe
3,-153AjTW5luZPK4omEujWA,Eisenhower once said Some people wanted ch...,3.000000,Jody Maroni's Sausage Kingdom
4,-1m9o3vGRA8IBPNvNqKLmA,Nice restaurant in Park MGM Las Vegas But I...,4.435388,Bavette's Steakhouse & Bar
...,...,...,...,...
6926,zwNC-Ow4eIMan2__bS9-rg,I can t complain My new favorite Mexican f...,4.052067,Taco Y Taco
6927,zwddv6HjTjDx9fOr3MFtwQ,Located on the 5th floor of the Imperial Pal...,2.461538,Quesadilla
6928,zwmKWhyyV-zejEUbQzBtHw,We heard of the XLBs here from Tony s review...,4.333333,A & K Chinese Restaurant
6929,zx_j6OuuHHa2afVoAZuLpA,My friend lives in Las Vegas but I m from L...,4.532258,what's Crepe


In [9]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(test_joined_restaurant.text.apply(lambda x: x.lower()))

KeyboardInterrupt: 

### User - User Simple TF-IDF and Cosine Similarity recommendation

In [ ]:
def user_recommendations(ID, df, record, k=10):
    """
    Recommends user based on a similarity dataframe
    Parameters
    ----------
    ID : str
        User ID (index of the similarity dataframe)
    df : pd.DataFrame
        Similarity dataframe, symmetric, with movies as indices and columns
    record : pd.DataFrame
        The record of original user and some other features used to define similarity
    k : int
        Amount of recommendations to return
    """
    ix = df.loc[:,ID].to_numpy().argpartition(range(-1,-k,-1))
    high_recomm = df.columns[ix[-1:-(k+2):-1]]
    high_recomm = high_recomm.drop(ID, errors='ignore')
    return pd.DataFrame(high_recomm).merge(record).head(k)

In [ ]:
# direct calculate the cosine similarity and generate the dataframe
cosine_sim = cosine_similarity(tfidf_matrix)

In [ ]:
user_user_df = pd.DataFrame(cosine_sim, columns=test.user_id, index=test.user_id)

In [ ]:
# given a user ID and the similarity matrix, recommend a list of similar user
user_id = '---1lKK3aKOuomHnwAkAow' # input user id
add_on = test[['user_id','categories']]

user_recommendations(user_id, user_user_df, add_on, k=5)

In [ ]:
test[test.user_id == user_id]

### Resraurant - Reataurant TF-IDF approach

In [ ]:
def calc_sim_user_rating(target_user_id, target_business_id, groupby_df, original_df):
    business_id_df = original_df[original_df.business_id == target_business_id].copy()
    
    target_user_tfidf = tfidf_matrix[groupby_df[groupby_df.user_id == target_user_id].index[0]].copy()
    
    dist_user = []
    for user_id in business_id_df.user_id.values:
        dist_user.append(cosine_similarity(tfidf_matrix[groupby_df[groupby_df.user_id==user_id].index[0]],
                                           target_user_vector)[0][0])
        
    business_id_df['similarity'] = dist_user    
    #Filtering technique = Calc percentile and filter percentiles > 80%
    business_id_df = business_id_df[business_id_df.rank(pct=True).similarity > .8]
     
    rating_diff = []
    for user_id in business_id_df.user_id.values:
        rating_diff.append(test_rest_user_id.stars_review[test_rest_user_id.user_id==user_id].values[0]
                            -np.average(city_train[city_train.user_id ==user_id].stars_review.values))
    test_rest_user_id['rating_diff'] = rating_diff
    
    return np.average(test_rest_user_id.rating_diff)

In [ ]:
test

In [ ]:
test[test.user_id == '---udAKDsn0yQXmzbWQNSw'].index[0]

In [ ]:
for i in tqdm(range(len(cities))):
    business_city = restaurants_business[restaurants_business.city == cities[i]]
    bus_rev_city = business_city.merge(reviews, on = 'business_id')

In [ ]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

results = {}

for idx, row in test.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], test['user_id'][i]) for i in similar_indices]

    results[row['user_id']] = similar_items[1:]
    
print('done!')

def item(id):
    return test.loc[test['user_id'] == id]['text'].tolist()[0].split(' - ')[0]

# Just reads the results out of the dictionary.
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")